In [1]:
import numpy as np
import cv2
from PIL import Image
import open3d as o3d
import json
import os
from tqdm import tqdm
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from matplotlib import rcParams
from shapely.geometry import MultiPoint, box

# from pyquaternion.quaternion import Quaternion
from pyquaternion import Quaternion
import os.path as osp
from nuscenes import NuScenes

# Utils for Lidar and Radar
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.geometry_utils import view_points, BoxVisibility
from nuscenes.utils.data_classes import LidarPointCloud
from nuscenes.utils.data_classes import RadarPointCloud
from nuscenes.scripts.export_2d_annotations_as_json import get_2d_boxes, post_process_coords
from utils import *

from typing import Tuple, List, Dict, Union

%matplotlib inline

In [2]:
#nusc = NuScenes(version='v1.0-mini', dataroot='/home/gus/Documents/AI_S/nuScenes/v1_0-mini', verbose=True)
nusc = NuScenes(version='v1.0-mini', dataroot='/home/sherlock/Documents/nuScenes/data', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.5 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


## Loop to create the features and targets file

In [3]:
# path with all the sample files
samples_path =  os.getcwd() + '/dataset/samples/'

# path to retrieve point clous features
features_path =  os.getcwd() + '/dataset/point_features_path/'

# Matrix to store the features' vectors
features_matrix = []
# Dictionary of targets
targets_dic = {'x_pos': [], 'y_pos': [], 'z_pos': [], 'width': [], 'lenght': [], 'height': [], 'yaw': []}

# extract all files in path
for root, dirs, files in os.walk(samples_path):
    
    # select each sample file
    for file_ in tqdm(files, desc = "Sample files"):    
        
        # load the current sample file
        data_json = load_file(samples_path + file_)
                
        # process each instance
        for idx in range(len(data_json['instance'])):
            
            # Create instance metadata
            annotation_metadata = data_json['instance'][idx]

            # Retrieve instance's features
            features_vec = parse_features_to_numpy(annotation_metadata['point_features_path'])
            
            # Store the current feature vector in the features' matrix
            features_matrix.append(list(features_vec))
            
            # Extract class targets
            pose = annotation_metadata['position_coord']
            dims = annotation_metadata['wlh_values']
            yaw = annotation_metadata['orientation_value']
            label = annotation_metadata['category']
            
            # Save X, Y and Z pose
            targets_dic['x_pos'].append(pose[0])
            targets_dic['y_pos'].append(pose[1])
            targets_dic['z_pos'].append(pose[2])
            
            # Store target size
            targets_dic['width'].append(dims[0])
            targets_dic['lenght'].append(dims[1])
            targets_dic['height'].append(dims[2])
            
            # Store target orientation
            targets_dic['yaw'].append(yaw)

            
# Parse features as a numpy array
features_matrix = np.array(features_matrix)

# Create a dataframe of features
df_features = pd.DataFrame(data = features_matrix)
# Create a dataframe of targets
df_targets = pd.DataFrame.from_dict(targets_dic)

# Concat both dataframes horizontally to create the final dataframe
df_detection = pd.concat([df_features, df_targets], axis=1)
df_detection.head()

df_detection.to_csv('dataset/3d_obj_detection.csv')

Sample files: 100%|██████████| 404/404 [00:00<00:00, 752.98it/s]


## Parsing info as a numpy array

In [10]:
# Parse features and targets to numpy array
features = df_features.to_numpy()
targets = df_targets.to_numpy()

print('The Features shape is {}'.format(features.shape))
print('The targets shape is {}'.format(targets.shape))

The Features shape is (1415, 640)
The targets shape is (1415, 7)


## Splitting and scaling the data

In [39]:
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler 

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=1)

print('Train set features size: \n {}'.format(X_train.shape))
print('Test set featires size: \n {}'.format(X_test.shape))
print('Train set targets size: \n {}'.format(y_train.shape))
print('Test set targets size: \n {}'.format(y_test.shape))

Train set features size: 
 (1132, 640)
Test set featires size: 
 (283, 640)
Train set targets size: 
 (1132, 7)
Test set targets size: 
 (283, 7)


In [49]:
scaler = MinMaxScaler()
scaler.fit(X_train) 
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [50]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge

clf = MultiOutputRegressor(Ridge(random_state = 123)).fit(X_train_scaled, y_train)

In [51]:
print('Training set score is: {:.2f}'.format(clf.score(X_train_scaled, y_train)))
print('Test set score is: {:.2f}'.format(clf.score(X_test_scaled, y_test)))

Training set score is: 0.61
Test set score is: 0.18


In [52]:
from sklearn import svm

clf = MultiOutputRegressor(svm.SVR(kernel='linear')).fit(X_train_scaled, y_train)

print('Training set score is: {:.2f}'.format(clf.score(X_train_scaled, y_train)))
print('Test set score is: {:.2f}'.format(clf.score(X_test_scaled, y_test)))

Training set score is: 0.38
Test set score is: 0.21


In [56]:
# Load the information from .txt file in JSON format
sample_path = os.getcwd()+'/dataset/samples/sample_841dd6709e9b4d7c9d6bf888f1fe6d7e_file.txt'

# This is a Dictionary of lists of dictionaries
data_json = load_file(sample_path)
data_json['instance'][0]

{'annotation_token': '4b8f390f86aa433cac6c4247f907565f',
 'sample_token': '841dd6709e9b4d7c9d6bf888f1fe6d7e',
 'camera_token': '39caaccc75dd4e38b8fe6f40e81f5621',
 'pointcloud_path': '/home/sherlock/Documents/3d_obj_detection/dataset/ann_points_lidar/pc_anno_4b8f390f86aa433cac6c4247f907565f.txt',
 'pcl_shape': [3, 794],
 'img_pc_path': '/home/sherlock/Documents/3d_obj_detection/dataset/ann_points_cam/img_pc_anno_4b8f390f86aa433cac6c4247f907565f.txt',
 'img_pcl_shape': [4, 794],
 'pcd_path': '/home/sherlock/Documents/3d_obj_detection/dataset/pcd_segmentation/pcd_segmentation_4b8f390f86aa433cac6c4247f907565f.pcd',
 'point_features_path': '/home/sherlock/Documents/3d_obj_detection/dataset/point_features/pcd_segmentation_4b8f390f86aa433cac6c4247f907565f.txt',
 'position_coord': [6.053757848521915, 4.1099025347326315, -0.899958760271085],
 'wlh_values': [1.772, 4.294, 1.597],
 'orientation_value': -177.91535813202847,
 'category': 'vehicle'}

In [64]:
from nuscenes.utils.geometry_utils import BoxVisibility

box_vis_level = BoxVisibility.ANY
    
sample_record = nusc.get('sample', data_json['instance'][0]['sample_token'])
cam = nusc.get('sample_data', data_json['instance'][0]['camera_token'])

_, boxes, camera_intrinsic = nusc.get_sample_data(sample_record['data'][cam['channel']], box_vis_level=box_vis_level,
                                                    selected_anntokens=[data_json['instance'][0]['annotation_token']])

https://github.com/nutonomy/nuscenes-devkit/blob/master/python-sdk/nuscenes/utils/data_classes.py#L572

https://github.com/nutonomy/nuscenes-devkit/blob/master/python-sdk/nuscenes/nuscenes.py#L1316

In [65]:
boxes[0]
camera_intrinsic

array([[1.26084744e+03, 0.00000000e+00, 8.07968245e+02],
       [0.00000000e+00, 1.26084744e+03, 4.95334427e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [69]:
boxes[0].corners()

array([[-0.52176648, -1.96608728, -1.91975839, -0.47543758,  1.96283816,
         0.51851735,  0.56484625,  2.00916706],
       [-0.18650868, -0.14021471,  1.45609862,  1.40980464, -0.24367553,
        -0.19738156,  1.39893177,  1.35263779],
       [ 4.37586692,  5.40142935,  5.39461757,  4.36905515,  7.87756762,
         8.90313004,  8.89631827,  7.87075584]])

In [ ]:
# Create box object out of predictions for visualziation 